In [ ]:
import pandas as pd
df_sample = pd.read_csv("/Users/htg/Desktop/Sydney_University/冒犯性检测/dataset/data_emnlp.csv", sep=",")

In [ ]:
#LLama COT
import pandas as pd
import re
from together import Together


# API key
api_key = "Your own key"


texts = df_sample.iloc[:, 0]

#创建together client
client = Together(api_key=api_key)

# 遍历数据集中的每一条记录进行判断
results = [] 
COT = []
for idx, line in enumerate(texts):
    # 构造 prompt，要求模型只返回 "1"（冒犯）或 "0"（不冒犯）
    prompt = f"""
            你是一个中文冒犯性语言检测专家
            请判断下面的文本是否具有冒犯性，文本可能包含谐音，表情等内容。
            这里的“冒犯性”主要指包含人身攻击、侮辱、歧视、仇恨言论或极端粗俗的内容。
            先一步步分析理由，然后在最后一行给出结论：
            - 如果冒犯，只输出 1
            - 如果不冒犯，只输出 0

            格式:
            思考过程: <逐步推理>
            结论: <0或1>

            文本如下：
            {line} 
            """
    # 接口
    response = client.chat.completions.create(
        model="meta-llama/Llama-3.3-70B-Instruct-Turbo",
        messages=[{"role": "user", "content": prompt}],
        max_tokens=3000,           
        temperature=0.1,
        top_p=0.9,
        top_k=5,
        repetition_penalty=1,
        stop=["<|eot_id|>", "<|eom_id|>"],
        stream=True            
    )
    
    # 输出模型返回的结果
    answer = ""
    for token in response:
        if hasattr(token, 'choices'):
            answer += token.choices[0].delta.content
    match_think = re.search(r"思考过程\s*[:：]\s*([\s\S]*?)结论", answer)
    think_text = match_think.group(1).strip() if match_think else ""
    match = re.search(r"结论\s*[:：]\s*([01])", answer)
    label = match.group(1) if match else ""
    results.append(label)
    COT.append(think_text)
    print(f"第 {idx + 1} 条数据COT：{think_text}")
    print(f"第 {idx + 1} 条数据判断结果：{label}")


result_df = pd.DataFrame({
    "Content": texts,
    "COT": COT,
    "Result": results
})


result_df.to_excel("./COT_result/LLama3.3_32B_results.xlsx", index=False)

In [ ]:
#LLama
import pandas as pd
import re
from together import Together


# API key
api_key = "Your own key"


texts = df_sample.iloc[:, 0]

#创建together client
client = Together(api_key=api_key)

# 遍历数据集中的每一条记录进行判断
results = [] 
for idx, line in enumerate(texts):
    # 构造 prompt，要求模型只返回 "1"（冒犯）或 "0"（不冒犯）
    prompt = f"""
                你是一个中文冒犯性语言检测专家
                请判断下面的文本是否具有冒犯性，文本可能包含谐音，表情等内容。
                这里的“冒犯性”主要指包含人身攻击、侮辱、歧视、仇恨言论或极端粗俗的内容。
                输出格式:
                结论: <0或1>

            文本如下：
            {line} 
            """
    # 接口
    response = client.chat.completions.create(
        model="meta-llama/Llama-3.3-70B-Instruct-Turbo",
        messages=[{"role": "user", "content": prompt}],
        max_tokens=500,           
        temperature=0.1,
        top_p=0.9,
        top_k=5,
        repetition_penalty=1,
        stop=["<|eot_id|>", "<|eom_id|>"],
        stream=True            
    )
    
    # 输出模型返回的结果
    answer = ""
    for token in response:
        if hasattr(token, 'choices'):
            answer += token.choices[0].delta.content
    
    match = re.search(r"结论\s*[:：]\s*([01])", answer)
    label = match.group(1) if match else ""
    results.append(label)
    print(f"第 {idx + 1} 条数据判断结果：{label}")


result_df = pd.DataFrame({
    "Content": texts,
    "Result": results
})


result_df.to_csv("./base_result/LLama_70b_results.csv", sep="\t", index=False)